In [ ]:
!pip install -q pyiceberg

In [ ]:
from pyiceberg.catalog.rest import RestCatalog
import logging

import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

CATALOG_URL = "http://lakekeeper:8181/catalog"
DEMO_WAREHOUSE = "demo"

CLIENT_ID = "spark"
CLIENT_SECRET = "2OR3eRvYfSZzzZ16MlPd95jhLnOaLM52"

catalog = RestCatalog(
    name="demo",
    warehouse="demo",
    uri=CATALOG_URL,
    credential=f"{CLIENT_ID}:{CLIENT_SECRET}",
    **{"oauth2-server-uri": "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token", "scope": "lakekeeper"},
)


In [ ]:
# Create a new namespace if it doesn't already exist
test_namespace = ("pyiceberg_namespace",)
if test_namespace not in catalog.list_namespaces():
    catalog.create_namespace(test_namespace)

In [ ]:
# Write data
test_table = ("pyiceberg_namespace", "my_table")
df = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "data": ["a", "b", "c"],
    }
)
pa_df = pa.Table.from_pandas(df)

if test_table in catalog.list_tables(namespace=test_namespace):
    catalog.drop_table(test_table)

table = catalog.create_table(
    test_table,
    schema=pa_df.schema,
)

table.append(pa_df)

In [ ]:
# Read data
table = catalog.load_table(test_table)
table.scan().to_pandas()